In [1]:
!pip install psycopg2-binary
import os
import csv
import requests
import psycopg2
from pytz import timezone
from datetime import datetime, timedelta
from dateutil import parser, relativedelta
import json
import collections
 

POSTGRES_USER = "postgres"
POSTGRES_PASSWORD = "admin1234"
POSTGRES_HOST = "mosaicai-sandbox.catnchrgoqwp.us-east-1.rds.amazonaws.com"
POSTGRES_PORT = "5432"
POSTGRES_DB_CONSOLE = "mosaic_console"
POSTGRES_DB_AI = "ai_logistics"
POSTGRES_DB_METERING = "metering"
POSTGRES_DB_USERMGM = "usermgm"
POSTGRES_DB_MONITOR = "mosaic_monitor"
DELIMETER = ""
LOGS = []
VOLUME_NAME = "/data"


def check_if_table_is_already_exist(connection, table_name):
    try:
        cursor = connection.cursor()
        cursor.execute("""SELECT COUNT(*) FROM information_schema.tables WHERE table_name = '{0}'""".format(table_name.replace('\'', '\'\'')))
        if cursor.fetchone()[0] == 1:
            return True
        return False
    except Exception as ex:
        LOGS.append("Exception (check_if_table_is_already_exist) : "+str(ex))
        print("Exception (check_if_table_is_already_exist) : "+str(ex))
    finally:
     if cursor:
        cursor.close()
        
        
def check_if_user_is_already_exists(user_bank_id, connection):
    try:
        cursor = connection.cursor()
        cursor.execute(" SELECT count(1) as record_count from "+POSTGRES_DB_USERMGM+".line_manager_information where user_bank_id='"+user_bank_id+"'")
        no_of_notebooks = cursor.fetchone()
        return not no_of_notebooks[0] == 0
    except Exception as ex:
        LOGS.append("Exception (check_if_user_is_already_exists) : "+str(user_bank_id)+":"+str(ex))
        print("Exception (check_if_user_is_already_exists) : "+str(ex))
    finally:
        if cursor:
            cursor.close()
        
def insert_line_manager(splits, connection):
    try:
        cursor = connection.cursor()
        cursor.execute("insert into "+POSTGRES_DB_USERMGM+".line_manager_information(user_bank_id,name,email_id,domain,line_manager_bank_id,created_at,created_by,updated_at,updated_by) values('"+splits[0]+"','"+splits[1]+"','"+splits[2]+"','"+splits[3]+"','"+splits[4]+"','"+datetime.now().strftime('%Y-%m-%d %H:%M:%S')+"','system','"+datetime.now().strftime('%Y-%m-%d %H:%M:%S')+"','system')")
        connection.commit()
        return True
    except Exception as ex:
        LOGS.append("Exception (insert_line_manager) : "+str(splits[0])+":"+str(ex))
        print("Exception (insert_line_manager) : "+str(ex))
    finally:
        if cursor:
            cursor.close()
            
            
def update_line_manager(splits, connection):
    try:
        cursor = connection.cursor()
        query = "update "+POSTGRES_DB_USERMGM+".line_manager_information set name='{0}',email_id='{1}',domain='{2}',line_manager_bank_id='{3}',updated_at='{4}' where user_bank_id='{5}'".format(splits[1],splits[2], splits[3], splits[4], datetime.now().strftime('%Y-%m-%d %H:%M:%S'), splits[0])
        cursor.execute(query)
        connection.commit()
        return True
    except Exception as ex:
        LOGS.append("Exception (update_line_manager) : "+str(splits[0])+":"+str(ex))
        print("Exception (update_line_manager) : "+str(ex))
    finally:
        if cursor:
            cursor.close()


     |████████████████████████████████| 3.0MB 15.6MB/s eta 0:00:01
You should consider upgrading via the 'pip install --upgrade pip' command.


/opt/conda/lib/python3.6/site-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
try:
    BANK_IDS = []
    UPDATE_CNT=0
    INSERT_CNT=0
    LOGS.clear()
    BANK_IDS.clear()
    connection_ai_logistics = psycopg2.connect(user=POSTGRES_USER,
                                               password=POSTGRES_PASSWORD,
                                               host=POSTGRES_HOST,
                                               port=POSTGRES_PORT,
                                               database=POSTGRES_DB_USERMGM)
    
    line_manager_information_exists = check_if_table_is_already_exist(connection_ai_logistics, "line_manager_information")
    
    if line_manager_information_exists:
        file1 = open(VOLUME_NAME+'/ocir_staff_master_'+datetime.now().strftime('%Y%m%d')+".csv", 'r')
        lines = file1.readlines()
        lines.pop(0)
        for line in lines:
            splits = line.split(DELIMETER)
            user_bank_id = splits[0]
            BANK_IDS.append(user_bank_id)
            is_exists = check_if_user_is_already_exists(user_bank_id, connection_ai_logistics)
            if is_exists:
                update_line_manager(splits, connection_ai_logistics)
                UPDATE_CNT = UPDATE_CNT+1
            else:
                insert_line_manager(splits, connection_ai_logistics)
                INSERT_CNT = INSERT_CNT+1
    else:
        LOGS.append("Table is not exists")
        print("Table is not exists")
    
except Exception as ex:
    LOGS.append("Exception : "+str(ex))
    print("Exception : ", ex)
finally:
    #BANK_IDS.append("10639405")
    duplicates = [{"Bank ID": item, "Count": count} for item, count in collections.Counter(BANK_IDS).items() if count > 1]
    print(duplicates)
    LOGS.append("Updated Records : "+str(UPDATE_CNT))
    LOGS.append("Inserted Records : "+str(INSERT_CNT))
    LOGS.append("Total CSV Records : "+str(UPDATE_CNT+INSERT_CNT))
    LOGS.append("Duplicate Records : "+str(duplicates))
    print(LOGS)
    if connection_ai_logistics:
        connection_ai_logistics.close()
        LOGS.append("Postgres Connection 'connection_ai_logistics' is closed")
        #print("\nPostgres Connection 'connection_ai_logistics' is closed") 
    log_dir = VOLUME_NAME+"/logs"
    if not os.path.exists(log_dir):
        os.makedirs(log_dir)
    with open(VOLUME_NAME+'/logs/ocir_staff_master_logs_'+datetime.now().strftime('%Y%m%d%H%M%S')+".txt", "w") as outfile:
        outfile.write("\n".join(LOGS))


[{'Bank ID': '1111113', 'Count': 2}]
['Updated Records : 11', 'Inserted Records : 0', 'Total CSV Records : 11', "Duplicate Records : [{'Bank ID': '1111113', 'Count': 2}]"]
